# 🚢 Titanic Data Analysis (Pandas 101)

Dans ce notebook, nous allons pratiquer **toutes les méthodes fondamentales de manipulation de données** avec **Pandas**, étape par étape, sur le dataset **Titanic**.



## 📌 1️⃣ Importer les librairies et le dataset

# 📑 Définition des colonnes — Titanic Dataset

| Colonne | Description |
|----------------|------------------------------------------------------------------|
| **PassengerId** | Identifiant unique du passager. |
| **Survived** | Statut de survie (0 = Non, 1 = Oui). |
| **Pclass** | Classe du billet (1 = 1ère, 2 = 2ème, 3 = 3ème). |
| **Name** | Nom complet du passager. |
| **Sex** | Sexe du passager (male / female). |
| **Age** | Âge du passager en années (peut contenir des valeurs manquantes). |
| **SibSp** | Nombre de frères/sœurs ou conjoints à bord. |
| **Parch** | Nombre de parents/enfants à bord. |
| **Ticket** | Numéro du billet. |
| **Fare** | Prix du billet payé (en livres sterling). |
| **Cabin** | Numéro de cabine (souvent manquant). |
| **Embarked** | Port d’embarquement (C = Cherbourg, Q = Queenstown, S = Southampton). |


## 🗂️ Détails

- **SibSp** : Frères, sœurs, conjoints.
- **Parch** : Parents, enfants.
- **Embarked** : Port où le passager est monté à bord.
- **Cabin** : Souvent manquant — certains passagers n’avaient pas de cabine attribuée ou enregistrée.
- **Fare** : Peut varier même pour la même classe en fonction de la cabine, du point d’embarquement et du statut de réservation.


👉 **Référence** : Ce dataset est tiré des archives originales du Titanic, largement utilisé pour apprendre la classification binaire et les méthodes d’exploration de données.



In [ ]:
import pandas as pd

# Charger Titanic
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

# Afficher les 5 premières lignes
df.head()

## 📌 2️⃣ Inspection de base

In [ ]:
# Informations générales
df.info()

# Statistiques descriptives
df.describe()

# Vérifier les valeurs manquantes
df.isnull().sum()


## 📌 3️⃣ Sélections : loc & iloc


- **`loc`** : Sélectionne des lignes et colonnes **par leurs étiquettes** (noms).
  - Ex. : `df.loc[2, 'Name']` → valeur de la colonne *Name* pour l’index 2.
- **`iloc`** : Sélectionne des lignes et colonnes **par position numérique**.
  - Ex. : `df.iloc[2, 3]` → même chose si *Name* est la 4ᵉ colonne.



In [ ]:
# loc : par étiquette
df.loc[:4, ['Name', 'Age', 'Sex']]

# iloc : par position
df.iloc[:5, [3, 5, 4]]


## 📌 4️⃣ Filtrer : conditions et query


- **Filtrage** : Extraire un sous-ensemble selon une **condition logique**.
  - Ex. : `df[df['Age'] > 30]` → lignes où l’âge est > 30.
- **`query`** : Équivalent SQL, plus lisible pour des conditions complexes.
  - Ex. : `df.query("Sex == 'female' & Age < 20")`.


In [6]:
# Filtrer les passagers adultes
adults = df[df['Age'] >= 18]

# Même filtrage avec query
adults_q = df.query("Age >= 18")


## 📌 5️⃣ Nettoyage : drop & drop_duplicates

- **`drop`** : Supprime des colonnes ou lignes.
  - Ex. : `df.drop(['Cabin'], axis=1)` → supprime la colonne *Cabin*.
- **`drop_duplicates`** : Supprime les lignes en double pour éviter les doublons.


In [7]:
# Supprimer une colonne inutile
df = df.drop(['Cabin'], axis=1)

# Vérifier doublons
df.duplicated().sum()

# Supprimer doublons s'il y en a
df = df.drop_duplicates()


## 📌 6️⃣ Trier : sort_values

- Trie les données par une ou plusieurs colonnes.
  - Ex. : `df.sort_values(by='Age', ascending=False)` → trie du plus vieux au plus jeune.


In [ ]:
# Trier par Age décroissant
df_sorted = df.sort_values(by='Age', ascending=False)
df_sorted.head()


## 📌 7️⃣ Grouper : groupby

- Regroupe les données **par une ou plusieurs colonnes clés** pour effectuer des opérations par groupe.
  - Ex. : `df.groupby(['Pclass', 'Sex'])` → regroupe par *classe* et *sexe*.



In [ ]:
# Grouper par classe et sexe
grouped = df.groupby(['Pclass', 'Sex'])
grouped.size()


## 📌 8️⃣ Agrégations : agg & aggregate

- Applique **une ou plusieurs fonctions d’agrégation** sur chaque groupe créé par `groupby`.
  - Ex. : `agg(['mean', 'median'])` → calcule la moyenne et la médiane.
  - Ex. : `agg({'Age': 'mean', 'Fare': 'max'})` → moyenne sur *Age*, max sur *Fare*.


In [ ]:
# Moyenne et médiane de l'âge
agg_age = grouped['Age'].agg(['mean', 'median'])
agg_age

# Agrégations multiples
agg_multi = grouped.agg({
    'Age': ['mean', 'std'],
    'Fare': ['mean', 'max']
})
agg_multi


## 📌 9️⃣ Table pivot : pivot_table

- **`pivot`** : Transforme les données en **tableau croisé** (index, colonnes, valeurs).
  - Simple mais nécessite que l’index/colonnes soient uniques.
- **`pivot_table`** : Plus flexible, permet de gérer plusieurs valeurs par groupe avec une fonction d’agrégation.
  - Ex. : `pivot_table(values='Fare', index='Pclass', columns='Sex', aggfunc='mean')`.


In [ ]:
# Survie par classe et sexe
pivot = df.pivot_table(
    values='PassengerId',
    index='Pclass',
    columns='Sex',
    aggfunc='count'
)
pivot


## 📌 🔀 1️⃣0️⃣ Format long : melt

- **Inverse de `pivot`** : Passe d’un format large à un format **long**.
  - Ex. : Plusieurs colonnes → une seule colonne *variable*, une *valeur*.


In [12]:
pivot_reset = pivot.reset_index()
melted = pivot_reset.melt(id_vars=['Pclass'], var_name='Sex', value_name='Count')
melted


,Pclass,Sex,Count
0,1,female,94
1,2,female,76
2,3,female,144
3,1,male,122
4,2,male,108
5,3,male,347


## 📌 🔗 1️⃣1️⃣ Fusionner : merge
- Combine plusieurs DataFrames **sur une clé commune** (équivalent SQL `JOIN`).
  - Ex. : `df.merge(df2, on='Pclass')`.


In [13]:
# Exemple : DataFrame des classes
df_classes = pd.DataFrame({
    'Pclass': [1, 2, 3],
    'Description': ['Upper', 'Middle', 'Lower']
})
df = df.merge(df_classes, on='Pclass', how='left')
df[['Pclass', 'Description']].drop_duplicates()


,Pclass,Description
0,3,Lower
1,1,Upper
9,2,Middle


## 📌 🧮 1️⃣2️⃣ Appliquer une fonction : apply et map
- **`apply`** : Applique une fonction sur chaque élément ou ligne/colonne.
  - Ex. : `df['Age'].apply(lambda x: x*2)`.
- **`map`** : Remplace ou transforme **chaque valeur individuelle** d’une série.
  - Ex. : `df['Sex'].map({'male': 'M', 'female': 'F'})`.



In [ ]:
# Exemple : créer groupe d'âge
df['AgeGroup'] = df['Age'].apply(lambda x: 'Child' if x < 18 else 'Adult')

# Remplacer Sexe par initiale (map)
df['SexInitial'] = df['Sex'].map({'male': 'M', 'female': 'F'})

df[['Name', 'Age', 'AgeGroup', 'Sex', 'SexInitial']].head()


## 📌 📊 1️⃣3️⃣ Compter valeurs : value_counts

In [ ]:
df['Survived'].value_counts()


## ✅ ✔️ Sauvegarde finale

In [16]:
df.to_csv('titanic_final.csv', index=False)
